In [1]:
!pip install ucimlrepo

In [2]:
!pip install bayesian-optimization

In [4]:
# importar datos
from ucimlrepo import fetch_ucirepo

# obtener datos
dermatology = fetch_ucirepo(id=33)

# datos como dataframe
X = dermatology.data.features
y = dermatology.data.targets

Esta base de datos contiene 34 atributos, 33 de los cuales tienen valores lineales y uno de ellos es nominal.

El diagnóstico diferencial de las enfermedades eritematoescamosas es un verdadero problema en dermatología. Todas ellas comparten las características clínicas del eritema y la descamación, con muy pocas diferencias. Las enfermedades de este grupo son la psoriasis, la dermatitis seborreica, el liquen plano, la pitiriasis rosada, la dermatitis crónica y la pitiriasis rubra pilaris. Por lo general, es necesaria una biopsia para el diagnóstico, pero lamentablemente estas enfermedades también comparten muchas características histopatológicas. Otra dificultad para el diagnóstico diferencial es que una enfermedad puede mostrar las características de otra enfermedad en la etapa inicial y puede tener las características características en las etapas siguientes. Los pacientes fueron evaluados clínicamente en primer lugar con 12 características. Después, se tomaron muestras de piel para la evaluación de 22 características histopatológicas. Los valores de las características histopatológicas se determinan mediante un análisis de las muestras bajo un microscopio.

En el conjunto de datos construido para este dominio, la característica de antecedentes familiares tiene el valor 1 si se ha observado alguna de estas enfermedades en la familia y 0 en caso contrario. La característica de edad simplemente representa la edad del paciente. A todas las demás características (clínicas e histopatológicas) se les asignó un grado en el rango de 0 a 3. Aquí, 0 indica que la característica no estaba presente, 3 indica la mayor cantidad posible y 1, 2 indican los valores intermedios relativos.

Los nombres y números de identificación de los pacientes se eliminaron recientemente de la base de datos.

##Modelo inicial

In [5]:
# Importar las bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [6]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Escalar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ajustar las etiquetas de las clases para que comiencen desde 0
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1

In [7]:
# Crear el modelo XGBoost para clasificación
model = XGBClassifier()
# Entrenar el modelo
model.fit(X_train, y_train_adjusted)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [8]:
# Realizar predicciones
y_pred_adjusted = model.predict(X_test)

# Ajustar las predicciones de nuevo a las etiquetas originales
y_pred = y_pred_adjusted + 1

# Evaluar el rendimiento del modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9454545454545454
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        34
           2       0.94      0.83      0.88        18
           3       1.00      0.91      0.95        22
           4       0.78      0.93      0.85        15
           5       0.94      1.00      0.97        15
           6       1.00      1.00      1.00         6

    accuracy                           0.95       110
   macro avg       0.94      0.95      0.94       110
weighted avg       0.95      0.95      0.95       110



*Accuracy para el Modelo Inicial es de :* **0.945**

## Modelo con ajuste de hiperparámetros (beyesian optimization)

In [9]:
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [21]:
# Definir la función objetivo para la optimización bayesiana
def xgb_evaluate(max_depth, learning_rate, n_estimators, gamma, min_child_weight, subsample, colsample_bytree):
    model = XGBClassifier(
        max_depth=int(max_depth),
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        gamma=gamma,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        eval_metric='mlogloss',
        random_state=42
    )

    cv_scores = cross_val_score(model, X_train, y_train_adjusted, cv=5, scoring='accuracy')
    return cv_scores.mean()

# Definir el espacio de búsqueda para los hiperparámetros
param_bounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'n_estimators': (50, 300),
    'gamma': (0, 5),
    'min_child_weight': (1, 10),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1)
}

# Realizar la optimización bayesiana
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=param_bounds,
    random_state=42,
    verbose=2
)

optimizer.maximize(init_points=10, n_iter=30)

# Obtener los mejores hiperparámetros
best_params = optimizer.max['params']
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.9726    | 0.6873    | 4.754     | 0.2223    | 7.191     | 2.404     | 89.0      | 0.529     |
| 2         | 0.9336    | 0.9331    | 3.006     | 0.2153    | 3.144     | 9.729     | 258.1     | 0.6062    |
| 3         | 0.9804    | 0.5909    | 0.917     | 0.09823   | 6.673     | 4.888     | 122.8     | 0.8059    |
| 4         | 0.9531    | 0.5697    | 1.461     | 0.1162    | 6.192     | 8.067     | 99.92     | 0.7571    |
| 5         | 0.9765    | 0.7962    | 0.2323    | 0.1862    | 4.194     | 1.585     | 287.2     | 0.9828    |
| 6         | 0.9804    | 0.9042    | 1.523     | 0.03832   | 7.79      | 4.961     | 80.51     | 0.7476    |
| 7         | 0.9804    | 0.5172    | 4.547     | 0.08505   | 7.638     | 3.805     | 180.0     | 0.7734    |
| 8       

In [22]:
# Crear el modelo XGBoost para clasificación
model = XGBClassifier(
    **best_params,
    random_state=42
)
# Entrenar el modelo
model.fit(X_train, y_train_adjusted)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5909124836035503, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.9170225492671691,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.09823025045826593,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=4.887505167779041, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=122,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [23]:
# Realizar predicciones y evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred + 1)

print("Accuracy después de la optimización:", accuracy)

Accuracy después de la optimización: 0.9636363636363636


*Accuracy para el Modelo con ajuste de hiperparámetros es de :* **0.963**